In [1]:
import csv
import datetime
import os
import yaml
from itertools import islice
from nupic.frameworks.opf.model_factory import ModelFactory

In [2]:
_NUM_RECORDS = 4300
_INPUT_FILE_PATH = './data/pepa_acc.csv'
_PARAMS_PATH = './params/model.yaml'

In [12]:
def createModel():
    with open(_PARAMS_PATH, "r") as f:
        modelParams = yaml.safe_load(f)
    return ModelFactory.create(modelParams)

def runHotgym(numRecords):
    model = createModel()
    model.enableInference({"predictedField": "pepa_acc_x"})
    with open(_INPUT_FILE_PATH) as fin:
        reader = csv.reader(fin)
        headers = reader.next()  # skip the header
        reader.next()
        reader.next()

        results = []
        for record in islice(reader, numRecords):
            modelInput = dict(zip(headers, record))
            modelInput["pepa_acc_x"] = float(modelInput["pepa_acc_x"])
            modelInput["pepa_acc_y"] = float(modelInput["pepa_acc_y"])
            modelInput["pepa_acc_z"] = float(modelInput["pepa_acc_z"])
            modelInput["timestamp"] = datetime.datetime.strptime(modelInput["time"], '%Y-%m-%d %H:%M:%S')
            
            result = model.run(modelInput)
            
            AnomalyScore = result.inferences['anomalyScore']
            bestPredictions = result.inferences["multiStepBestPredictions"]
            fiveStep = bestPredictions[5]
            result = (fiveStep, AnomalyScore)
            print "5-step: {:16} \t anomaly score: {:16}".format(*result)
            results.append(result)
        return results

In [13]:
if __name__ == "__main__":
    runHotgym(_NUM_RECORDS)

ValueError: No predicted field was enabled! Did you call enableInference()?